In [1]:
import sys
sys.executable

'/Users/niksaz/anaconda2/envs/sound/bin/python'

In [2]:
from laughter_classification.sspnet_data_sampler import SSPNetDataSampler

FRAME_LENGTH_SECONDS = 0.5

sampler = SSPNetDataSampler('corpus/')
data = sampler.create_sampled_df(FRAME_LENGTH_SECONDS, save_path='samples.csv')

saving df:  samples.csv


In [4]:
data.head(25)

,V0,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V140,V141,V142,V143,V144,V145,V146,V147,IS_LAUGHTER,SNAME
0,903.469993,22.110552,13.151943,46.213083,-17.533179,23.288871,-17.945700,4.478985,-16.021022,-8.827005,...,8.985123e+07,4.989764e+07,4.625960e+07,3.625273e+07,5.107908e+07,7.357580e+07,6.407850e+07,7.800024e+06,0.0,S1584
1,999.305112,23.627986,0.111502,43.626124,-20.154165,12.678496,-21.751440,-1.124484,-19.371105,-7.697882,...,2.144139e+08,2.310223e+08,3.068134e+08,2.213448e+08,2.040658e+08,1.592024e+08,1.295517e+08,2.404268e+07,0.0,S1584
2,1044.186757,35.652589,-1.837158,33.768778,-27.015343,9.455086,-22.789318,-7.763146,-19.690869,-4.318618,...,2.443953e+08,2.470301e+08,3.161120e+08,2.298107e+08,2.128765e+08,1.661035e+08,1.351158e+08,2.656138e+07,0.0,S1584
3,1073.704985,36.993531,-7.366114,25.173819,-25.322474,3.895748,-24.025736,-6.938407,-20.043033,-0.449823,...,8.844644e+08,6.865517e+08,4.733031e+08,4.908453e+08,3.562223e+08,1.688878e+08,1.204767e+08,3.649412e+07,0.0,S1584
4,1076.620915,39.017817,-11.557705,18.714980,-26.330274,4.989787,-18.932455,-9.438138,-15.903813,3.718074,...,9.439361e+08,7.555011e+08,5.229335e+08,5.567003e+08,4.156222e+08,2.205994e+08,1.500357e+08,5.049134e+07,0.0,S1584
5,1043.189105,20.742445,-0.316556,21.908099,-33.418329,12.352806,-18.662568,-7.130745,-16.506184,5.269059,...,2.602697e+09,1.719883e+09,8.422123e+08,6.415686e+08,3.725757e+08,2.812581e+08,1.320899e+08,4.180392e+07,0.0,S1584
6,1048.920863,6.376005,8.230949,25.988574,-35.820946,3.864877,-21.808773,-4.387632,-19.596030,0.379147,...,3.224770e+09,1.797646e+09,1.029810e+09,8.712102e+08,5.430676e+08,4.009626e+08,1.663391e+08,4.469680e+07,0.0,S1584
7,1034.103227,5.644245,3.849416,33.427645,-31.333013,0.233446,-16.815885,-0.325079,-21.143798,-4.942796,...,2.595622e+09,1.417278e+09,9.758874e+08,7.202293e+08,4.619874e+08,4.145889e+08,1.588686e+08,3.696932e+07,0.0,S1584
8,995.621771,-1.909144,2.695838,55.443860,-32.991052,-0.126695,-19.138405,3.964841,-24.811939,-17.698905,...,2.482918e+09,1.335233e+09,9.182979e+08,6.711759e+08,4.114660e+08,3.394379e+08,1.120452e+08,1.447846e+07,0.0,S1584
9,952.869735,18.554375,-0.570314,62.683327,-25.129590,-3.356810,-22.969112,3.985010,-26.553290,-28.253279,...,3.648106e+08,2.028988e+08,2.198786e+08,3.022872e+08,3.173742e+08,1.708140e+08,5.917018e+07,9.504342e+06,1.0,S1584


In [8]:
def to_categorical_vectors(y, num_classes):
    y = np.array(y, dtype='int')
    input_shape = y.shape
    y = y.ravel()
    n = y.shape[0]
    categorical = np.zeros((n, num_classes), dtype='float32')
    categorical[np.arange(n), y] = 1
    output_shape = input_shape + (num_classes,)
    categorical = np.reshape(categorical, output_shape)
    return categorical

In [19]:
import random
import librosa
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc

from laughter_classification.visualization import plot_ROC_curve
from laughter_prediction.feature_extractors import LibrosaExtractor
from laughter_prediction.predictors import RnnPredictor

chunks = 22
# Default parameters in Librosa library
N_MFCC = 20   
N_MELS = 128
features = N_MFCC + N_MELS

index = data['SNAME'].unique()
train_index, test_index = train_test_split(index)
train_data = np.array(data[data['SNAME'].isin(train_index)].drop(columns='SNAME'))
test_data = np.array(data[data['SNAME'].isin(test_index)].drop(columns='SNAME'))

(217427, 149)
(15048, 149)


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [ ]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)


training_data = [
    ("The dog ate the apple".split(), ["DET", "NN", "V", "DET", "NN"]),
    ("Everybody read that book".split(), ["NN", "V", "DET", "NN"])
]
word_to_ix = {}
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
print(word_to_ix)
tag_to_ix = {"DET": 0, "NN": 1, "V": 2}

# These will usually be more like 32 or 64 dimensional.
# We will keep them small, so we can see how the weights change as we train.
EMBEDDING_DIM = 6
HIDDEN_DIM = 6

In [ ]:
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)
        self.hidden = self.init_hidden()

    def init_hidden(self):
        # Before we've done anything, we dont have any hidden state.
        # Refer to the Pytorch documentation to see exactly
        # why they have this dimensionality.
        # The axes semantics are (num_layers, minibatch_size, hidden_dim)
        return (torch.zeros(1, 1, self.hidden_dim),
                torch.zeros(1, 1, self.hidden_dim))

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, self.hidden = self.lstm(
            embeds.view(len(sentence), 1, -1), self.hidden)
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

In [ ]:
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

# See what the scores are before training
# Note that element i,j of the output is the score for tag j for word i.
# Here we don't need to train, so the code is wrapped in torch.no_grad()
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)
    print(tag_scores)

for epoch in range(300):  # again, normally you would NOT do 300 epochs, it is toy data
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Also, we need to clear out the hidden state of the LSTM,
        # detaching it from its history on the last instance.
        model.hidden = model.init_hidden()

        # Step 2. Get our inputs ready for the network, that is, turn them into
        # Tensors of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)

        # Step 3. Run our forward pass.
        tag_scores = model(sentence_in)

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()

# See what the scores are after training
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)

    # The sentence is "the dog ate the apple".  i,j corresponds to score for tag j
    # for word i. The predicted tag is the maximum scoring tag.
    # Here, we can see the predicted sequence below is 0 1 2 0 1
    # since 0 is index of the maximum value of row 1,
    # 1 is the index of maximum value of row 2, etc.
    # Which is DET NOUN VERB DET NOUN, the correct sequence!
    print(tag_scores)

In [21]:
extractor = LibrosaExtractor()
features = extractor.extract_features('/Users/niksaz/University/speech-ml-2019/2/homework/corpus/data/S2763.wav')
print(features)

            0          1          2          3          4          5    \
0   1130.052966  59.804751 -24.188314   6.759751 -22.642437 -11.349979   
1   1102.118035  67.359458 -19.510806  15.376573 -20.681987  -7.748538   
2   1095.463665  65.467280 -23.795737  10.027540 -19.907009 -12.355085   
3   1084.072479  66.758237 -23.309363  12.459292 -20.634887 -15.749771   
4   1088.183836  58.298530 -21.737682  18.719988 -15.796714 -10.661384   
5   1073.899819  57.523585 -21.092532  18.725367 -15.884023  -9.529497   
6   1070.819135  51.467568 -23.156605  22.866770 -18.892777 -12.915802   
7   1049.519946  76.129197 -21.996905  31.799868 -15.211159 -13.120435   
8   1056.572013  88.730549 -22.058374  36.659142 -13.998237 -16.911113   
9   1054.975505  79.274556 -36.108519  35.645814 -12.901140 -19.417350   
10  1017.274949  76.923851 -34.857947  35.741469  -6.984779 -13.505964   
11  1019.324751  61.172379 -39.126861  29.860043  -5.815897  -7.851868   
12   970.522862  67.072898 -33.504962 